In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc


#from PIL import Image
import PIL.Image as Image
import torchvision.transforms as transforms
to_tensor = transforms.ToTensor()
import torchvision
import torch.nn as nn
from functools import partial
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

!pip install torchinfo
from torchinfo import summary




2023-05-19 14:39:33.004862: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


You should consider upgrading via the '/Users/ilaydailerten/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
current_dir = 'TONIC_TMA4_R1C2/'
#os.mkdir(new_dir)

cdir = !ls $current_dir

for n in cdir:
    if n[-5:] == '.tiff':
        newn = n.replace(".tiff", ".tif" )
        os.rename(current_dir+n, current_dir+newn)



In [3]:

cdir = !ls $current_dir


i=0
for n in cdir:
    image_dir = current_dir+n
    if i==0:
        im = Image.open(image_dir)
        #im = im.resize([2024,2024])
        image = to_tensor(im)
    else:
        im = Image.open(image_dir) 
        #im = im.resize([224,224])
        next_channel = to_tensor(im)
        image = torch.cat((image,next_channel),0)
    i += 1
print(image.shape)


torch.Size([38, 2048, 2048])


In [4]:
resnet_models = {18: torchvision.models.resnet18,
                 34: torchvision.models.resnet34,
                 50: torchvision.models.resnet50,
                 101: torchvision.models.resnet101,
                 152: torchvision.models.resnet152}


In [5]:
class Resnet_multichannel(nn.Module):
    def __init__(self, pretrained=True, encoder_depth=34, num_in_channels=4):
        super().__init__()
        
        if encoder_depth not in [18, 34, 50, 101, 152]:
            raise ValueError(f"Encoder depth {encoder_depth} specified does not match any existing Resnet models")
            
        model = resnet_models[encoder_depth](pretrained)
        
        print(model.conv1)
        
        ##For reference: layers to use (in order):
        # conv1, bn1, relu, maxpool, layer1, layer2, layer3, layer4, avgpool, fc
        
        # This increases the number of in channels for our network
        self.conv1 = self.increase_channels(model.conv1, num_in_channels)
        
        self.bn1 = model.bn1
        self.relu = model.relu
        self.maxpool = model.maxpool
        self.layer1 = model.layer1
        self.layer2 = model.layer2
        self.layer3 = model.layer3
        self.layer4 = model.layer4
        self.avgpool = model.avgpool
        self.fc = model.fc
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
        
    def increase_channels(self, m, num_channels=None, copy_weights=0):
        """
        takes as input a Conv2d layer and returns the a Conv2d layer with `num_channels` input channels
        and all the previous weights copied into the new layer.
        """
        # number of input channels the new module should have
        new_in_channels = num_channels if num_channels is not None else m.in_channels + 1
        
        print(m.in_channels)
        print(new_in_channels)
       
        
        bias = False if m.bias is None else True
        
        # Creating new Conv2d layer
        new_m = nn.Conv2d(in_channels=new_in_channels, 
                          out_channels=m.out_channels, 
                          kernel_size=m.kernel_size, 
                          stride=m.stride, 
                          padding=m.padding,
                          bias=bias)
        
        # Copying the weights from the old to the new layer
        new_m.weight.data[:, :m.in_channels, :, :] = m.weight.clone()
        
        #Copying the weights of the `copy_weights` channel of the old layer to the extra channels of the new layer
        print(new_in_channels - m.in_channels)
        for i in range(new_in_channels - m.in_channels):
            channel = m.in_channels + i
            new_m.weight.data[:, channel:channel+1, :, :] = m.weight[:, copy_weights:copy_weights+1, : :].clone()
        new_m.weight = nn.Parameter(new_m.weight)

        return new_m
    


In [6]:
def get_arch(encoder_depth, num_in_channels):
    """
    Returns just an architecture which can then be called in the usual way.
    For example:
    resnet34_4_channel = get_arch(34, 4)
    model = resnet34_4_channel(True)
    """
    return partial(Resnet_multichannel, encoder_depth=encoder_depth, num_in_channels=num_in_channels)

In [7]:
resnet50_mchannel = get_arch(50, 38)
model = resnet50_mchannel(True) 
print("New input channels : ", model.conv1.in_channels)

/Users/ilaydailerten/opt/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/Users/ilaydailerten/opt/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
3
38
35
New input channels :  38


In [8]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            
feature_extract = True
set_parameter_requires_grad(model, feature_extract)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
input_size = 2024 #224

In [9]:
print(summary(model))


Layer (type:depth-idx)                   Param #
Resnet_multichannel                      --
├─Conv2d: 1-1                            (119,168)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─Bottleneck: 2-1                   --
│    │    └─Conv2d: 3-1                  (4,096)
│    │    └─BatchNorm2d: 3-2             (128)
│    │    └─Conv2d: 3-3                  (36,864)
│    │    └─BatchNorm2d: 3-4             (128)
│    │    └─Conv2d: 3-5                  (16,384)
│    │    └─BatchNorm2d: 3-6             (512)
│    │    └─ReLU: 3-7                    --
│    │    └─Sequential: 3-8              (16,896)
│    └─Bottleneck: 2-2                   --
│    │    └─Conv2d: 3-9                  (16,384)
│    │    └─BatchNorm2d: 3-10            (128)
│    │    └─Conv2d: 3-11                 (36,864)
│    │    └─BatchNorm2d: 3-12            (128)
│    │    └

In [10]:
# image = torch.unsqueeze(image, 1)
uimage = torch.unsqueeze(image, 0)
model.eval()

Resnet_multichannel(
  (conv1): Conv2d(38, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1

In [11]:
out = model(uimage)
print(out)

tensor([[-0.1966,  0.2150]], grad_fn=<AddmmBackward0>)
